# Create OCEAN Data NFT
Quickly create OCEAN Data NFTs by querying blockchain data from Dune and Flipside.

**Steps:**
1. Get Data
2. Store Data
3. Create Data NFT

In [2]:
import json
import pandas as pd
import os
import time
from dotenv import load_dotenv
import requests
import subprocess
from flipside import Flipside
from ocean_lib.web3_internal.utils import connect_to_network
from ocean_lib.example_config import get_config_dict
from ocean_lib.ocean.ocean import Ocean
from brownie.network import accounts, chain

# Load variables from .env file
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
FLIPSIDE_API_KEY = os.getenv("FLIPSIDE_API_KEY")

## 1. Get Data
### Flipside

* SDK Docs: https://docs.flipsidecrypto.com/flipside-api/get-started

In [2]:
# Query Flipside using their Python SDK
def query_flipside(sql):
    flipside = Flipside(FLIPSIDE_API_KEY, "https://api-v2.flipsidecrypto.xyz")
    results = flipside.query(sql)
    results_df = pd.DataFrame(results.rows, columns=results.columns).drop(columns=["__row_index"])
    return results_df

In [3]:
sql = """
SELECT
  date(block_timestamp) as dt,
  count(distinct tx_hash) as tx_ct
FROM ethereum.core.fact_transactions
WHERE block_timestamp >= GETDATE() - interval'30 days'
GROUP BY 1
order by 1 asc
"""
results_df = query_flipside(sql)
results_df

,dt,tx_ct
0,2023-07-20T00:00:00.000Z,333141
1,2023-07-21T00:00:00.000Z,1110882
2,2023-07-22T00:00:00.000Z,1004374
3,2023-07-23T00:00:00.000Z,977305
4,2023-07-24T00:00:00.000Z,1036300
5,2023-07-25T00:00:00.000Z,990249
6,2023-07-26T00:00:00.000Z,1029348
7,2023-07-27T00:00:00.000Z,998420
8,2023-07-28T00:00:00.000Z,1014601
9,2023-07-29T00:00:00.000Z,991161


### Dune

* API Docs: https://dune.com/docs/api/

In [ ]:
# Query Dune Analytics via API
def query_dune(q):
    url = f"https://api.dune.com/api/v1/query/{q}/results?api_key={DUNE_API_KEY}"
    response = requests.get(url)
    results_json = json.loads(response.text)["result"]["rows"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [ ]:
dune_query_id = 2847682
results_df = query_dune(dune_query_id)
results_df

## 2. Store Data
### Add File to GitHub

*Note: This is a quick & easy solution to host the data, but can also add file to IPFS, Arweave, GCP, AWS, etc...*

In [4]:
# Create File from Query results
results_df.to_csv('query_results.csv', index=False)

In [5]:
# Store file in GitHub repo
def git_add_commit_push(commit_message):
    try:
        subprocess.run(['git', 'add', '.'])
        subprocess.run(['git', 'commit', '-m', commit_message])
        subprocess.run(['git', 'push'])
        print("Git add, commit, and push successful!")
    except Exception as e:
        print("An error occurred:", e)

# Replace 'Your commit message here' with your desired commit message
commit_message = 'Your commit message here'

git_add_commit_push(commit_message)

[main c7ed66e] Your commit message here
 2 files changed, 21 insertions(+), 5 deletions(-)
Git add, commit, and push successful!


remote: 
remote: GitHub found 17 vulnerabilities on PrimoData/ocean_data_nfts's default branch (9 high, 5 moderate, 3 low). To find out more, visit:        
remote:      https://github.com/PrimoData/ocean_data_nfts/security/dependabot        
remote: 
To https://github.com/PrimoData/ocean_data_nfts
   5c5ab23..c7ed66e  main -> main


## 3. Create Data NFT
### Add Data NFT to the OCEAN Protocol Marketplace

* Marketplace: https://market.oceanprotocol.com/
* SDK Docs: https://docs.oceanprotocol.com/developers/ocean.py/install

In [3]:
# Connect to Blockchain
connect_to_network("polygon-main")
config = get_config_dict("polygon-main")
ocean = Ocean(config)

In [4]:
# Connect to Wallet
accounts.clear()
private_key = os.getenv('PRIVATE_KEY')
wallet = accounts.add(private_key)
print('MATIC Balance:', wallet.balance() / 1e18 )

MATIC Balance: 15.864497269367353


In [5]:
# Create Data NFT
name = "Data NFT Test 16"

# Link to File
url = "https://raw.githubusercontent.com/PrimoData/ocean_data_nfts/main/query_results.csv"

# Set Gas Fees
priority_fee = chain.priority_fee
max_fee = (chain.base_fee * 2) + priority_fee
gas_limit = int((priority_fee + max_fee) /1e5)
print("priority_fee:", priority_fee, "max_fee:", max_fee, "gas_limit:", gas_limit)

# Configure Transaction Variables
tx_dict = {
    "from": wallet,
    "priority_fee": priority_fee,
    "max_fee": max_fee,
    "gas_limit": gas_limit,
    "allow_revert": True,
}

# Publish Data NFT
(data_nft, datatoken, ddo) = ocean.assets.create_url_asset(
    name, 
    url, 
    tx_dict
)

time.sleep(30)

# Set Price to Free
datatoken.create_dispenser(tx_dict)

print("Data NFT published:")
print(f"  data_nft: symbol={data_nft.symbol()}, address={data_nft.address}")
print(f"  datatoken: symbol={datatoken.symbol()}, address={datatoken.address}")
print(f"  did={ddo.did}")
print(f"  Link: https://market.oceanprotocol.com/asset/{ddo.did}")


priority_fee: 30000000000 max_fee: 136212032580 gas_limit: 1662120
Transaction sent: 0xd5c2cc8ca6dccadaaec3ac6723c8c5dd989f3e897e9b46634750191c337b81ea
  Max fee: 136.21203258 gwei   Priority fee: 30.0 gwei   Gas limit: 1662120   Nonce: 65
  ERC721Factory.createNftWithErc20 confirmed   Block: 46504040   Gas used: 1595694 (96.00%)   Gas price: 82.252346515 gwei



INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x04464a7213be991ff9428d6893a2730aa7de7177142ca62a72dc7e82cef297b7bbe3fae5a91165a876413011925e55ff346f9ba62b5bee7e6731637be241453f43286e88842b2e2bd2073b5e030dbd3e78581be81e1d91c75e1fee7e33f42fc28c1e1b78cd20ec5da1a5dac6c727bfdda193ff6e7a668d13586e4851d9a811b1e43b06847a901c047aa3227ddb796eb15e1c74769813ece5f4ebfeb005f6a34cf7cfc08ad59cfc1553343138a257e233d9de3d8943cfe61aa0e5ad5df33243afb3a1e9d2343f01a6c526e2e23a972b1d5a0c75ee49f31cc032bc4bb27a144c11b4800fd85d66109aa3c84211eba115bfaffb522796ac3e57d5d74519102df93a42136c98df2e8376389d3c035d48540003bd150362999982a931fcb34d3ce08337a8cbb503a8856adb8db56df873ef49f5176fd9e9cd6e7aeeb48e0f1af9f4ac26f1205b9850cbb7cdb43da7260ef6d3, encryptedEndpoint https://v4.provider.polygon.oceanprotocol.com/api/services/encrypt?chainId=137
INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x04204f78d7d959fe6402d8efbae

Transaction sent: 0x0f54fb55b60da9a540defbb4b5d68c9e7a813ec11e489efac88f8081188cb5d4
  Max fee: 136.21203258 gwei   Priority fee: 30.0 gwei   Gas limit: 1662120   Nonce: 66
  ERC721Template.setMetaData confirmed   Block: 46504043   Gas used: 212471 (12.78%)   Gas price: 83.71147739 gwei

Transaction sent: 0xf11c2481a15ab7ee74ced79618a4cb7e03e687cf819b2b95f6e8eeef58cd0ada
  Max fee: 136.21203258 gwei   Priority fee: 30.0 gwei   Gas limit: 1662120   Nonce: 67
  ERC20Template.createDispenser confirmed   Block: 46504072   Gas used: 283122 (17.03%)   Gas price: 86.381964226 gwei

Data NFT published:
  data_nft: symbol=Data NFT Test 16, address=0x7A4892c31991Ae4C0b9df7b72d9D67030cb9FFC4
  datatoken: symbol=DT1, address=0x223Cb17F54C7C4a2A3377BC5DD305Ebb71def2AF
  did=did:op:cf32001c1d8ba46b5b7d29f50c28dcef319d902f405f300b9bb393808b59d3f8
  Link: https://market.oceanprotocol.com/asset/did:op:cf32001c1d8ba46b5b7d29f50c28dcef319d902f405f300b9bb393808b59d3f8
